In [1]:
%matplotlib inline
import pyurdme
import numpy

In [2]:
class Simple_Diffusion(pyurdme.URDMEModel):
    """ Initial condition is a delta function at and off-center point. 
        The solution should be a Gaussian, and periodic boundary conditions
        should allow it to wrap. """

    def __init__(self):

        pyurdme.URDMEModel.__init__(self,name="simple_diffusion")

        D = 0.01
        A = pyurdme.Species(name="A",diffusion_constant=D)

        self.add_species([A])

        # A unit square
        self.mesh = pyurdme.URDMEMesh.generate_unit_square_mesh(40,40, periodic=True)

        # Place the A molecules in the voxel nearest the center of the square
        self.set_initial_condition_place_near({A:100000},point=[0.1,0.1])

        self.timespan(numpy.linspace(0,5,1000))

In [3]:
model = Simple_Diffusion()

In [4]:
%time result = model.run()

CPU times: user 932 ms, sys: 112 ms, total: 1.04 s
Wall time: 14.7 s


In [5]:
#result.display('A',-1)

In [6]:
def display(result, species, timepoint, opacity=1.0, wireframe=True, width=500):
    data = result.get_species(species,timepoint,concentration=True)
    fun = pyurdme.DolfinFunctionWrapper(result.model.mesh.get_function_space())
    vec = fun.vector()
    nd = data.shape[0]
    if nd == len(vec):
        for i in range(nd):
            vec[i]=data[i]
    else:
        v2d= result.get_v2d()
        for i in range(len(vec)):
            vec[i]=data[i]
    fun.display(opacity=opacity, wireframe=wireframe, width=width)

In [10]:
display(result, 'A', 200)

In [8]:
model.mesh